In [9]:
import pandas as pd
import numpy as np

from datetime import datetime

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots


from sqlalchemy import create_engine


In [10]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'
TABLE_SCHEMA = 'diploma'
ENGINE = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')

query_users = f'''SELECT * FROM {TABLE_SCHEMA}.users'''
query_geo_daily_report = f'''SELECT * FROM {TABLE_SCHEMA}.geo_daily_report'''

# время от установки до первого платежа

In [16]:
# время от установки до первого платежа
data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

start_time = datetime.now()

first_pay_time = pd.read_sql(f''' 
WITH
-- разделяем когорты по типу трафика и по условию
    user_types AS (
        SELECT
            *
        FROM {TABLE_SCHEMA}.users 
        WHERE
            1=1
            AND project_id = '{data_project_id}' 
            AND event_date >='{data_start}'
            AND event_date < '{data_end}'
),
last_date AS (
-- даты всех логинов игроков
    SELECT
        DISTINCT
            appsflyer_id,
            (event_date)::date AS ld       
    FROM
        user_types
    WHERE 1=1
    AND payments IS NOT NULL
    AND payments = 1
),
first_date AS (
-- дата первого входа
    SELECT
        install_date::date AS fd,
        appsflyer_id
    FROM
        user_types
    WHERE 1=1
    AND payments IS NOT NULL
    AND payments = 1
)
-- джоин все на все и вычитаем интервалы ld и fd
    SELECT
       last_date.appsflyer_id,
       ld - fd AS install_to_first_pay
    FROM
        first_date
    JOIN
        last_date ON last_date.appsflyer_id = first_date.appsflyer_id   		

''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
first_pay_time

execution time: 0:00:00.376874


,appsflyer_id,install_to_first_pay
0,1651241121340-2553095920057507326,0
1,1650764266895-7653803464843509212,4
2,1651084919281-6450730314891177953,0
3,1651069022924-7580205085528549047,0
4,1651001531024-290434569188491305,1
...,...,...
660,1649020108312-3020462346959226096,24
661,1649797328206-5116002045611716572,18
662,1651295054299-5719566577996852726,0
663,1651258387196-5627116178419769732,1


In [18]:
# среднее время от установки до первого платежа
first_pay_time.install_to_first_pay.mean()

7.861654135338346

In [20]:
# Кол-во всех первых покупок за период
first_pay_time.install_to_first_pay.sum()

5228